In [31]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define some parameters for the simulation
n_spans = 1000        # number of spans
length_span = 10   # length of each span (km)
beta1 = 0           # chromatic dispersion (s^2/km)
gamma = 0           # nonlinear coefficient (1/W/km)
alpha = 0.0         # attenuation coefficient (dB/km)
DGD_max = 10e-12    # maximum differential group delay (s)
P_max = 1           # maximum power (W)

# Generate a random PMD vector for each span
PMD = np.random.uniform(0, DGD_max, size=n_spans)

# Generate a random Jones matrix for each span
J = np.zeros((2, 2, n_spans), dtype=np.complex128)
for i in range(n_spans):
    theta = np.random.uniform(0, np.pi/2)
    phi = np.random.uniform(0, np.pi)
    J[:, :, i] = np.array([[np.exp(1j*phi)*np.cos(theta), -np.sin(theta)],
                            [np.sin(theta), np.exp(-1j*phi)*np.cos(theta)]])

# Generate the input signal as a random polarization state
S_in = np.ones((2,))
S_in /= np.linalg.norm(S_in)

# Define a function that applies the Jones matrix to a polarization state
def apply_J(S, J):
    return np.matmul(J, S)

# Apply the PMD and Jones matrices to the input signal
S = S_in
S_traj = np.zeros((3, n_spans+1))
S_traj[:, 0] = np.array([np.real(S[0]), np.real(S[1]), np.imag(S[0])])
for i in range(n_spans):
    J_i = J[:, :, i]
    S = apply_J(S, J_i)
    DGD_i = PMD[i]
    S *= np.exp(1j*np.pi/2*DGD_i*beta1*np.array([1, -1]))
    
    S_traj[:, i+1] = np.array([np.real(S[0]), np.real(S[1]), np.imag(S[0])])

# # Plot the Poincare sphere
# fig = plt.figure
# def plot_poincare_sphere(S_traj, color='blue'):
#     fig = plt.figure()
#     ax = fig.add_subplot(111, projection='3d')
#     ax.plot(S_traj[0, :], S_traj[1, :], S_traj[2, :], color=color)
#     ax.set_xlim(-1, 1)
#     ax.set_ylim(-1, 1)
#     ax.set_zlim(-1, 1)
#     ax.set_xlabel('X')
#     ax.set_ylabel('Y')
#     ax.set_zlabel('Z')
#     plt.show()
    
import plotly.graph_objs as go
from plotly.subplots import make_subplots

def plot_poincare_sphere(S_traj):
    fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])

    fig.add_trace(
        go.Scatter3d(
            x=S_traj[0,:],
            y=S_traj[1,:],
            z=S_traj[2,:],
            mode='markers',
            marker=dict(size=3),
            line=dict(color='blue', width=2)
        )
    )

    fig.update_layout(
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z'),
            aspectratio=dict(x=1, y=1, z=1),
            camera=dict(
                up=dict(x=0, y=0, z=1),
                eye=dict(x=1.5, y=1.5, z=1.5)
            )
        ),
        margin=dict(l=0, r=0, b=0, t=0)
    )

    fig.show()

In [32]:
#%matplotlib inline
plot_poincare_sphere(S_traj)
#plot_poincare_sphere(S_in,color='red')

In [ ]:
def multiSecPMDchannel_v1(Ei, L, secLen, DGD, β1):
    """
    Multi-section MDM SDM channel model
    
    Parameters
    ----------
    Ei : ndarray
        N-dim input field envelope.
    L : float
        Total fiber length.
    Lspan : int
        Span length.
    secPerSpan : int
        Number of sections per spans.
    a : float
        Initial value of the uniform distribution interval [a,b] in dB.
    b : float
        Final value of the  uniform distribution interval [a,b] in dB.
    β1 : float
        Uncoupled group delays per unit length.
    D : float
        Uncoupled mode-dependent CD parameter per unit length.
    Fc : float
        Central optical frequency.
        
    Returns
    -------
    Eo : ndarray
        N-dim output field envelope.
 
    References
    -----------
    [1] Choutagunta, K.,; Kahn, J. M. (2017). Dynamic Channel Modeling for 
    Mode-Division Multiplexing. Journal of Lightwave Technology, 35(12), 
    2451–2463. https://doi.org/10.1109/JLT.2017.2656821
    
    """                       
    # c  = 299792458   # speed of light [m/s](vacuum)      
    nSec = L//secLen
                   
    Nfft = len(Ei) # fft size

    ω = 2 * np.pi * Fs * fftfreq(Nfft)
    ω = ω.reshape(ω.size, 1)

    try:
        Nmodes = Ei.shape[1]
    except IndexError:
        Nmodes = 1
        Ei = Ei.reshape(Ei.size, Nmodes)

    ω = np.tile(ω, (1, Nmodes)) # frequency vector
       
    Eo = Ei.copy()       
    G = np.eye(Nmodes, dtype='complex')
    
    indSec = 0
    # propagate the signal through N static sections
    for span in tqdm(range(Nspans)):
        for i in range(secPerSpan):         
            Hsec_dyn = np.exp( - 1j * β1 * ω  * secLen) # linear operator in the frequency domain
        
            Usec = sp.stats.unitary_group.rvs(Nmodes) # generate random Nmodes-by-Nmodes unitary matrix U
            Vsec = sp.stats.unitary_group.rvs(Nmodes) # generate random Nmodes-by-Nmodes unitary matrix V
                        
            # calculate eq.(2) in [1]
            Eo = np.dot(Eo, Vsec.conj().T)    
            Eo = ifft(fft(Eo, axis=0) * Hsec_dyn * Hsec_static, axis=0)
            Eo = np.dot(Eo, Usec)
            indSec +=1          
    return Eo